<a href="https://colab.research.google.com/github/boshuaiYu/huggingface_tutorial/blob/Transformers/HuggingFace1_%E4%BD%BF%E7%94%A8%E6%A1%88%E4%BE%8B%E5%9F%BA%E7%A1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Mon Dec 18 09:13:51 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install transformers

In [4]:
import transformers

In [5]:
!pip install transformers[sentencepiece]

### **1.情感分析**

#### **1.1 pipeline所支持的任务**

In [8]:
from transformers.pipelines import SUPPORTED_TASKS

In [ ]:
for k, v in SUPPORTED_TASKS.items():
  print(k + "  |  " + v["type"])

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

#### **1.2 pipeline直接构建**

In [16]:
classifier

In [17]:
classifier("I've been waiting for a HuggingFace course my whole life.")

[{'label': 'POSITIVE', 'score': 0.9598046541213989}]

In [24]:
classifier("I've been waiting for a HuggingFace course my whole life, but I hate it so much!")

[{'label': 'NEGATIVE', 'score': 0.9982317090034485}]

In [25]:
classifier(["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"])

[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [28]:
classifier("我喜欢春天")

[{'label': 'POSITIVE', 'score': 0.6589778661727905}]

In [29]:
classifier("我不喜欢夏天")

[{'label': 'NEGATIVE', 'score': 0.6316300630569458}]

In [31]:
classifier("我喜欢秋天，但我不喜欢冬天")

[{'label': 'NEGATIVE', 'score': 0.8265683054924011}]

In [ ]:
classifier1 = pipeline("sentiment-analysis",model="BAAI/bge-reranker-large")

In [36]:
classifier1("我喜欢秋天，但我不喜欢冬天")

[{'label': 'LABEL_0', 'score': 0.276503324508667}]

#### **1.3 先预加载模型, 再构建pipeline**

In [ ]:
from transformers import *

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-large")

In [ ]:
model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-large")

In [ ]:
tokenizer

In [47]:
classifier2 = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [49]:
classifier2("我喜欢秋天，但我不喜欢冬天")

[{'label': 'LABEL_0', 'score': 0.276503324508667}]

In [50]:
classifier2.model.device

device(type='cpu')

In [14]:
input = "我喜欢秋天，但我不喜欢冬天"
inputs = tokenizer(input, return_tensors="pt")
model(**inputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.9619]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [51]:
classifier3 = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [52]:
classifier3("我喜欢秋天，但我不喜欢冬天")

[{'label': 'LABEL_0', 'score': 0.2765016555786133}]

In [53]:
classifier3.model.device

device(type='cuda', index=0)

### 2.零样本分类

In [ ]:
classifier_1 = pipeline("zero-shot-classification", device=0)

In [ ]:
classifier_1(
    "I am a phd in HIT",
    candidate_labels=["education", "politics", "business"],
)

### 3.文本生成

#### 3.1默认模型

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation")

In [59]:
generator("In this course, we will teach you how to",max_length=100,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to use Mantle to create a high quality object pipeline.\n\nSo let\'s see if it will work or not!\n\nLet\'s get started…\n\nCreating the object pipeline…\n\n<!DOCTYPE html > <!--...--> < script src = "../mantle-3.11 " > < / script > < script src = "../mantle-3.11/main.js " > <!--'},
 {'generated_text': 'In this course, we will teach you how to easily set up and manage your own custom video editing software for more efficient, interactive editing. How to create videos from scratch? With the help of your favorite software, you can.\n\nHow to program video files into editing programs\n\nThis course will teach you how to easily program video file format from scratch. You will want to take this course with you to see how easy it is to create video-making videos.\n\nHow to program'}]

#### 3.2指定模型

In [ ]:
generator1 = pipeline("text-generation", model="distilgpt2") ## distilgpt2

In [61]:
generator1("In this course, we will teach you how to",max_length=30,num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to write a set of useful features.\n\n\n\nLearn to Create a List of Things About Common'},
 {'generated_text': 'In this course, we will teach you how to use the new technologies to increase productivity of the new business model and how they are designed.\n\n'}]

### 4.填充文本

In [ ]:
unmask = pipeline("fill-mask",model="bert-base-uncased")

In [64]:
unmask("This course will teach you all about [MASK] models.", top_k=5)

[{'score': 0.33661195635795593,
  'token': 2535,
  'token_str': 'role',
  'sequence': 'this course will teach you all about role models.'},
 {'score': 0.08371733874082565,
  'token': 2449,
  'token_str': 'business',
  'sequence': 'this course will teach you all about business models.'},
 {'score': 0.05718289688229561,
  'token': 1996,
  'token_str': 'the',
  'sequence': 'this course will teach you all about the models.'},
 {'score': 0.04392540454864502,
  'token': 2115,
  'token_str': 'your',
  'sequence': 'this course will teach you all about your models.'},
 {'score': 0.026684772223234177,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': 'this course will teach you all about fashion models.'}]

### 5.命名实体识别

In [ ]:
ner = pipeline("ner",grouped_entities=True)

In [66]:
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")
## PER: Person, ORG: Organization, LOC: Location
## grouped_entities=True 以告诉pipeline将对应于同一实体的句子部分重新组合在一起
## 这里模型正确地将“Hugging”和“Face”分组为一个组织，即使名称由多个词组成

[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

### 6.问答系统

In [ ]:
QA = pipeline("question-answering")

In [68]:
QA(question="Where do I work?",context="My name is Sylvain and I work at Hugging Face in Brooklyn")
# 此pipeline通过从提供的上下文中提取信息来工作；它不会凭空生成答案

{'score': 0.6949754953384399, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

### 7.文本摘要

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/config.json
Model config BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/pytorch_model.bin
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1
}



In [ ]:
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

### 8.翻译

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

In [ ]:
translator("Ce cours est produit par Hugging Face.")

[{'translation_text': 'This course is produced by Hugging Face.'}]

### 9.偏见与局限

In [ ]:
unmasker1 = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker1("This man works as a [MASK].")
print([r["token_str"] for r in result])

result = unmasker1("This woman works as a [MASK].")
print([r["token_str"] for r in result])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']
['nurse', 'maid', 'teacher', 'waitress', 'prostitute']
